In [0]:
dbutils.fs.mounts()

In [0]:
%fs
ls "/mnt/j2dadlscontainer/raw_datasets"

In [0]:
reviews_df = spark.read.format("csv").option("header", True).option("inferSchema", True).load("dbfs:/mnt/j2dadlscontainer/raw_datasets/deliverAgentReviews.csv")
reviews_df.display()

In [0]:
def replace_spaces(df):
    for col in df.columns:
        df = df.withColumnRenamed(col, (col.replace(" ", "_")).lower())
    return df

In [0]:
reviews_df = replace_spaces(reviews_df)
display(reviews_df)

In [0]:
reviews_df_temp = reviews_df.select("agent_name", "location", "customer_service_rating").dropDuplicates()

In [0]:
reviews_df_temp.createOrReplaceTempView("reviews")

In [0]:
%sql
select * from reviews;

find out the second highest rated city for each agent

In [0]:
%sql

select agent_name, location, customer_service_rating as rating, 
row_number() over (partition by agent_name, location order by customer_service_rating desc) as rn
from reviews;

1. using sub-query

In [0]:
%sql
select agent_name, location, rating from
(select agent_name, location, customer_service_rating as rating, 
row_number() over (partition by agent_name, location order by customer_service_rating desc) as rn
from reviews)
where rn = 2;

In [0]:
%sql
select agent_name, location, customer_service_rating as rating, 
row_number() over (partition by agent_name, location order by customer_service_rating) as rn
from reviews;

In [0]:
%sql
select agent_name, location, rating from
(select agent_name, location, customer_service_rating as rating, 
row_number() over (partition by agent_name, location order by customer_service_rating) as rn
from reviews)
where rn = 1;

pyspark window functions

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [0]:
df_temp = (
    reviews_df_temp
    .withColumn(
        "rn", 
        F.row_number()
        .over(
            Window
            .partitionBy("agent_name", "location")
            .orderBy(
                F.col("customer_service_rating").desc()
            )
        )
    )
).filter(F.col("rn") == 2).display()


In [0]:
df_temp.filter(df_temp.rn == 2).display()

In [0]:
windowSpec = Window.partitionBy("agent_name", "location").orderBy(F.col("customer_service_rating").asc())

df_temp = (
    reviews_df_temp
    .withColumn(
        "rn",
        F.row_number().over(windowSpec)
    )
).filter(F.col("rn") == 1).display()

In [0]:
cars_df = spark.read.format("csv").option("header", True).option("inferSchema", True).load("dbfs:/mnt/j2dadlscontainer/raw_datasets/car_price_dataset.csv")
cars_df.display()

for each Brand | model car I want to extract the second least owner count

In [0]:
windowSpec = Window.partitionBy("Brand", "Model").orderBy(F.col("Mileage").asc())

cars_df_temp = (
    cars_df
    .withColumn(
        "rn",
        F.dense_rank().over(windowSpec)
    )
).select("Brand", "Model", "Mileage")

In [0]:
cars_df_temp.filter(F.col("rn") == 2).display()

In [0]:
# Example

emp = [(1,"Smith",-1,"2018","10","M",3000), \
    (2,"Rose",1,"2010","20","M",4000), \
    (3,"Williams",1,"2010","10","M",1000), \
    (4,"Jones",2,"2005","10","F",2000), \
    (5,"Brown",2,"2010","40","",-1), \
      (6,"Brown",2,"2010","50","",-1) \
  ]
empColumns = ["emp_id","name","superior_emp_id","year_joined", \
       "emp_dept_id","gender","salary"]

empDF = spark.createDataFrame(data=emp, schema = empColumns)


dept = [("Finance",10), \
    ("Marketing",20), \
    ("Sales",30), \
    ("IT",40) \
  ]
deptColumns = ["dept_name","dept_id"]
deptDF = spark.createDataFrame(data=dept, schema = deptColumns)

In [0]:
empDF.display()

In [0]:
deptDF.display()

In [0]:
# temp views
empDF.createOrReplaceTempView("employee")
deptDF.createOrReplaceTempView("department")

In [0]:
%sql
select e.emp_id, e.name, d.dept_name
from employee e
join department d
on e.emp_dept_id = d.dept_id;

In [0]:
df_temp = (
    empDF
    .join(
        deptDF,
        empDF.emp_dept_id == deptDF.dept_id,
        how="left"
    )
).select("emp_id", "name", "dept_name")
df_temp.display()